# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set. 

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation 

    * Il migliore MAE di train

    * Il MAE di test 


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni. 

In [1]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [2]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

Numero di combinazioni: 56


In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training. 

In [7]:
# Carica il dataset Wine Quality White
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True)
X = data.data
y = data.target.astype(float)  # Assicura che il target sia float per la regressione

#1
def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    """
    Addestra un modello di regressione lineare con eventuale PCA e regolarizzazione L2.
    """
    if hyperparams['use_pca']:
        # ...
        if hyperparams['pca_standardize']: #applico standardizzazione
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_val = scaler.transform(X_val)

        #applico la pca
        pca = PCA(n_components= hyperparams['pca_components'])
        X_train = pca.fit_transform(X_train)
        X_val = pca.transform(X_val)

    if hyperparams['data_standardize']: #applico standardizzazione
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)
        
    # Aggiunge il termine costante ai dati
    X_train = np.c_[np.ones(X_train.shape[0]), X_train]
    X_val = np.c_[np.ones(X_val.shape[0]), X_val]
    
    parametri = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train
        
    if hyperparams['use_regularization']: # regolarizzazione
        lamda=hyperparams['reg_lambda']
        I=np.identity(X_train.shape[1]) #matrice identita'
        
        parametri = np.linalg.inv( (X_train.T @ X_train) + lamda * I) @ X_train.T @ y_train
    else:
        parametri = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train
        

    # Calcolo predizioni
    train_prediction = X_train @ parametri
    val_prediction = X_val @ parametri
    
    # Calcola il MAE
    mae_train = np.mean(np.abs(train_prediction - y_train))
    mae_val = np.mean(np.abs(val_prediction- y_val))
    
    return mae_train, mae_val

#2
# Scrivo una funzione che utilizza la pipeline definita al punto 1, testando tutte le configurazioni possibili presenti in configs

def test_pipeline(X, y, hyperparams):
    # Divido il dataset in training set e test set
    train_porzione = 0.8  
    
    num_train = int(train_porzione * X.shape[0])

    X_train_val = X[:num_train]
    y_train_val = y[:num_train]

    X_test = X[num_train:]
    y_test = y[num_train:]
    print("Training set: ",X_train_val.shape)
    print("Test set: ",X_test.shape)
    

    # Divido il training set in training set effettivo e validation set
    num_train = int(train_porzione * X_train_val.shape[0])

    X_train = X_train_val[:num_train]  
    y_train = y_train_val[:num_train]

    X_val = X_train_val[num_train:] 
    y_val = y_train_val[num_train:]
    #controllo la suddivisione dei set
    print("Training set effettivo: ",X_train.shape)
    print("Validation set: ",X_val.shape)
    
    # Trova la configurazione di iperparametri migliore
    top_mae_val=float('inf')   
    top_mae_train=0  
    top_config=  None  
    print("\nCerco la migliore configurazione degli iperparametri")
    for config in hyperparams: 
        #creo una copia dei dati per ogni configurazione 
        X_train_copy = X_train.copy()
        X_val_copy = X_val.copy()

        mae_train, mae_val = pipeline(X_train_copy, y_train, X_val_copy, y_val, config)

        #salvo il mae migliore tra le varie configurazioni 
        if mae_val < top_mae_val:
            top_mae_val = mae_val
            top_mae_train = mae_train
            top_config = config

    print("Migliore configurazione trovata:")
    print(top_config)

    return top_config, top_mae_train, top_mae_val

#3
def pipeline_best_mae(X, y, hyperparams):
    #suddivido dataset in training set e in test set
    train_porzione = 0.8
    num_train = int(train_porzione * X.shape[0])

    X_train = X[:num_train]
    y_train = y[:num_train]

    X_test = X[num_train:]
    y_test = y[num_train:]

    #Rialleno il modello sul training set completo 
    top_config = None
    top_config, top_mae_train, top_mae_val = test_pipeline(X, y, hyperparams)
    
    #Calcolo il MAE sul test set
    mae_train, mae_test = pipeline(X_train, y_train, X_test, y_test, top_config) 
    
    return top_config, top_mae_train, top_mae_val, mae_test

config,mae_train,mae_val,mae_test=pipeline_best_mae(X,y,configs)

# Stampa  risultati
print("\n\nStampa dei risultati")
print("Migliore configurazione:",config)
print("Migliore MAE Training: ",mae_train)
print("Migliore MAE Validation :", mae_val)
print("Migliore MAE Test :", mae_test)

/usr/local/lib/python3.11/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Training set:  (3918, 11)
Test set:  (980, 11)
Training set effettivo:  (3134, 11)
Validation set:  (784, 11)

Cerco la migliore configurazione degli iperparametri
Migliore configurazione trovata:
{'use_pca': True, 'pca_standardize': False, 'pca_components': 10, 'data_standardize': True, 'use_regularization': True, 'reg_lambda': 10}


Stampa dei risultati
Migliore configurazione: {'use_pca': True, 'pca_standardize': False, 'pca_components': 10, 'data_standardize': True, 'use_regularization': True, 'reg_lambda': 10}
Migliore MAE Training:  0.6061245872009969
Migliore MAE Validation : 0.5714080179441893
Migliore MAE Test : 0.5456079524655569
